In [11]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 968.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 78.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 67.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [38]:
import boto3
import librosa
import math
import os
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [ ]:
#how to access files on s3

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
bucket = s3.Bucket("music-wav-charts")

i = 0
for obj in bucket.objects.all(): #resource
    if(obj.key[len(obj.key)-3:]) == "png":
        print(obj.key)
        i+=1
    #file = s3_client.download_file('music-wav-files', obj.key, 'audio.wav')
    #audio, sfreq = librosa.load('audio.wav')
    #print(audio)
print(i)

In [39]:
#how to access files on s3

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
bucket = s3.Bucket("music-wav-charts-2")

i = 0
for obj in bucket.objects.all(): #resource
    if(obj.key[len(obj.key)-3:]) == "wav":
        print(obj.key)
        i+=1
    #file = s3_client.download_file('music-wav-files', obj.key, 'audio.wav')
    #audio, sfreq = librosa.load('audio.wav')
    #print(audio)
print(i)

genres_original/blues/blues.00000.wav
genres_original/blues/blues.00001.wav
genres_original/blues/blues.00002.wav
genres_original/blues/blues.00003.wav
genres_original/blues/blues.00004.wav
genres_original/blues/blues.00005.wav
genres_original/blues/blues.00006.wav
genres_original/blues/blues.00007.wav
genres_original/blues/blues.00008.wav
genres_original/blues/blues.00009.wav
genres_original/blues/blues.00010.wav
genres_original/classical/classical.00000.wav
genres_original/classical/classical.00001.wav
genres_original/classical/classical.00002.wav
genres_original/classical/classical.00003.wav
genres_original/classical/classical.00004.wav
genres_original/classical/classical.00005.wav
genres_original/classical/classical.00006.wav
genres_original/classical/classical.00007.wav
genres_original/classical/classical.00008.wav
genres_original/classical/classical.00009.wav
genres_original/classical/classical.00010.wav
genres_original/country/country.00000.wav
genres_original/country/country.00

In [43]:
#USED TO CREATE THE DATASET

SAMPLE_RATE = 22050
DURATION = 30 #measured in seconds

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
bucket = s3.Bucket("music-wav-files")

SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION
def save_mfcc(n_mfcc=13, n_fft=2048, 
              hop_length=512, num_segments = 5):
    
    with open('data.json', 'w') as fp:
        #dictionary to store data
        data = {
            "mapping": ["blues", "classical", "country", "disco",
                       "hiphop", "metal", "pop", "reggae", "rock"],
            "mfcc": [],
            "labels": []
        }

        num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
        expected_num_mfcc_vectors_per_seg = math.ceil(num_samples_per_segment / hop_length)
        j = 1
        k = 1
        for obj in bucket.objects.all(): #resource
            if j == 100:
                k+=1
                j = 1
                
            if(obj.key[len(obj.key)-3:]) == "wav":
                file = s3_client.download_file('music-wav-files', obj.key, 'audio.wav')
                print("\nProcessing {}".format(obj.key))


                #load audio file
                signal, sr = librosa.load('audio.wav', sr=SAMPLE_RATE)

                #process segments extracting mfcc/storing data
                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment

                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                               sr = sr,
                                               n_fft = n_fft,
                                               n_mfcc = n_mfcc,
                                               hop_length=hop_length
                                               )

                    mfcc = mfcc.T

                    #store mfcc for segment if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_seg:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(k - 1)
                        print("{}, segment: {}".format(obj.key, s))

                j+=1


        json.dump(data, fp)

        return data


mfcc_dataset = save_mfcc(num_segments=10)
print("done processing")

0 genres_original/blues/blues.00000.wav

Processing genres_original/blues/blues.00000.wav
genres_original/blues/blues.00000.wav, segment: 0
genres_original/blues/blues.00000.wav, segment: 1
genres_original/blues/blues.00000.wav, segment: 2


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[ 0.00732422  0.01660156  0.00762939 ... -0.04373169 -0.05706787
 -0.04086304] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.02481079 -0.02005005 -0.0357666  ...  0.00918579  0.03964233
  0.09030151] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[ 0.06256104  0.02435303 -0.00405884 ...  0.2854004   0.29364014
  0.25228882] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: Fu

genres_original/blues/blues.00000.wav, segment: 3
genres_original/blues/blues.00000.wav, segment: 4
genres_original/blues/blues.00000.wav, segment: 5
genres_original/blues/blues.00000.wav, segment: 6
genres_original/blues/blues.00000.wav, segment: 7
genres_original/blues/blues.00000.wav, segment: 8
genres_original/blues/blues.00000.wav, segment: 9
1 genres_original/blues/blues.00001.wav

Processing genres_original/blues/blues.00001.wav
genres_original/blues/blues.00001.wav, segment: 0
genres_original/blues/blues.00001.wav, segment: 1
genres_original/blues/blues.00001.wav, segment: 2
genres_original/blues/blues.00001.wav, segment: 3


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[0.00341797 0.00430298 0.00137329 ... 0.15249634 0.13171387 0.10726929] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[ 0.15649414  0.15270996  0.11993408 ... -0.1390686  -0.14138794
 -0.15228271] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.17068481 -0.18624878 -0.17926025 ... -0.01953125 -0.01815796
 -0.01568604] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWar

genres_original/blues/blues.00001.wav, segment: 4
genres_original/blues/blues.00001.wav, segment: 5
genres_original/blues/blues.00001.wav, segment: 6
genres_original/blues/blues.00001.wav, segment: 7
genres_original/blues/blues.00001.wav, segment: 8
genres_original/blues/blues.00001.wav, segment: 9
2 genres_original/blues/blues.00002.wav

Processing genres_original/blues/blues.00002.wav
genres_original/blues/blues.00002.wav, segment: 0
genres_original/blues/blues.00002.wav, segment: 1
genres_original/blues/blues.00002.wav, segment: 2


...

genres_original/rock/rock.00009.wav, segment: 3
genres_original/rock/rock.00009.wav, segment: 4
genres_original/rock/rock.00009.wav, segment: 5
genres_original/rock/rock.00009.wav, segment: 6
genres_original/rock/rock.00009.wav, segment: 7
genres_original/rock/rock.00009.wav, segment: 8
genres_original/rock/rock.00009.wav, segment: 9
10 genres_original/rock/rock.00010.wav

Processing genres_original/rock/rock.00010.wav
genres_original/rock/rock.00010.wav, segment: 0
genres_original/rock/rock.00010.wav, segment: 1


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.14608765 -0.09909058 -0.06234741 ...  0.02719116  0.01199341
  0.00463867] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.01608276 -0.0335083  -0.05020142 ... -0.03634644 -0.0480957
 -0.06082153] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.07034302 -0.07928467 -0.08926392 ... -0.13519287 -0.059021
  0.01184082] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: Futur

genres_original/rock/rock.00010.wav, segment: 2
genres_original/rock/rock.00010.wav, segment: 3
genres_original/rock/rock.00010.wav, segment: 4
genres_original/rock/rock.00010.wav, segment: 5
genres_original/rock/rock.00010.wav, segment: 6
genres_original/rock/rock.00010.wav, segment: 7
genres_original/rock/rock.00010.wav, segment: 8
genres_original/rock/rock.00010.wav, segment: 9


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.07244873 -0.10449219 -0.09103394 ... -0.02828979 -0.02307129
 -0.01260376] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.01553345 -0.02008057 -0.02011108 ... -0.37850952 -0.25576782
 -0.1942749 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Pass y=[-0.06207275  0.02822876  0.01205444 ...  0.18289185  0.17041016
  0.14746094] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:49: Fu

done processing


In [ ]:
#load the dataset
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["mfcc"]) #X is the MFCC - input
    Y = np.array(data["labels"]) #Y is the labels (0-8) - output. 
    return X,Y #inputs, targets

In [ ]:
def prepare_datasets(test_size, validation_size):
    #load data
    X, Y = load_data('data.json')
    
    #create the train/test split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)
    
    #create the train/validation split
    X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=validation_size)

    #3d array -> (130, 13, 1) see 12:00
    X_train = X_train[...,np.newaxis] #4d array -> (num_samples, 130, 13, 1)
    X_validation = X_validation[...,np.newaxis]
    X_test = X_test[...,np.newaxis]
    
    return X_train, X_validation, X_test, Y_train, Y_validation, Y_test

In [ ]:
def build_model(input_shape):
    #create model that you will pass the mfccs arrays into. once you pass them through these layers,
    #you should get a 1d array which allows you to map a given mfcc array to a music genre
    model = keras.Sequential()
    
    #1st conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', 
                        input_shape=input_shape)) 
    #32 filters, kernel grid size: 3x3, relu activation function, same input_shape as X_train
    
    model.add(keras.layers.MaxPool2D( (3,3), strides=(2,2), padding='same' ))
    #we maxpool so we can immediately downsample the input
    #stride is the rate at which the neural network moves over the MFCCs
    
    model.add(keras.layers.BatchNormalization())
    #a process that normalizes the activations in the current layer and the applications
    #that gets present to the subsequent layer. speeds up the model much faster because the numbers
    #are smaller.
    
    #2nd conv layer - same as the 1st
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', 
                                 input_shape=input_shape))
    
    model.add(keras.layers.MaxPool2D( (3,3), strides=(2,2), padding='same' ))
    model.add(keras.layers.BatchNormalization())
    
    
    #3rd conv layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu', 
                                 input_shape=input_shape))
    #we want to change the kernel to 2x2
    
    model.add(keras.layers.MaxPool2D( (2,2), strides=(2,2), padding='same' ))
    #we want to change the stride to 2x2
    
    model.add(keras.layers.BatchNormalization())
    
    #flatten the output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    
    #output layer - want to flatten the 2d array into a 1d array.
    #the 1d array is the mapping that associates a given mfcc array to a specific genre, as discussed
    #in the dataset design section in this project's report
    model.add(keras.layers.Dense(10,activation='softmax'))
    #we will be returning this 1d array
    
    return model

In [32]:
correct = 0

def predict(model, X, Y):
    global correct
    X = X[np.newaxis, ...]
    
    #prediction = [ [0.1, 0.2, ...] ]
    prediction = model.predict(X) #X -> (1, 130, 13, 1)
    
    #extract index (genre)
    predicted_index = np.argmax(prediction, axis=1) # [3]
    print("Expected index: {}, Predicted index: {}".format(
    Y, predicted_index))
    
    if predicted_index == Y:
        correct+=1
    
    
if __name__ == "__main__":
    #create train, validation, and test sets
    X_train, X_validation, X_test, Y_train, Y_validation, Y_test = prepare_datasets(0.25, 0.2)
    #X_train trains the dataset. 25% of data.json is used for this
    #X_validate verifies that the information is accurate. 20% of the data is used for this.
    #X_test is to verify the model works. 55% of the data is used for this.

    #build the CNN network
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]) 
    #X_train.shape[0] is the number of samples.
    #.shape[1] is the number of time bins
    #.shape[2] is the number of MFCCs
    #.shape[3] is the number of channels/depth
    
    model = build_model(input_shape)
    
    #compile the network
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    #train the CNN
    model.fit(X_train, Y_train, validation_data=(X_validation, Y_validation),
              batch_size=32,
              epochs=30 #train the model for 30 iterations
             )
    
    #evaluate the CNN on the test set
    test_error, test_accuracy = model.evaluate(X_test, Y_test,
                                              verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))
    
    
    #make predictions on a sample
    for i in range(100, 2248):
        X = X_test[i]
        Y = Y_test[i]
        predict(model, X,Y)
    print("Accuracy Rate: " + str(correct/2147) )

    #The accuracy rate is 67%. For a model which uses less than 50% of a dataset to train and validate,
    #this is pretty good.


Epoch 1/30
169/169 [==============================] - 9s 52ms/step - loss: 2.4855 - accuracy: 0.2305 - val_loss: 1.9807 - val_accuracy: 0.3281
Epoch 2/30
169/169 [==============================] - 9s 51ms/step - loss: 1.9202 - accuracy: 0.3593 - val_loss: 1.6722 - val_accuracy: 0.4222
Epoch 3/30
169/169 [==============================] - 9s 52ms/step - loss: 1.7268 - accuracy: 0.4119 - val_loss: 1.5316 - val_accuracy: 0.4526
Epoch 4/30
169/169 [==============================] - 9s 53ms/step - loss: 1.5949 - accuracy: 0.4423 - val_loss: 1.4484 - val_accuracy: 0.4867
Epoch 5/30
169/169 [==============================] - 9s 51ms/step - loss: 1.5050 - accuracy: 0.4745 - val_loss: 1.3923 - val_accuracy: 0.5081
Epoch 6/30
169/169 [==============================] - 9s 51ms/step - loss: 1.4335 - accuracy: 0.4960 - val_loss: 1.3460 - val_accuracy: 0.5267
Epoch 7/30
169/169 [==============================] - 10s 61ms/step - loss: 1.3839 - accuracy: 0.5103 - val_loss: 1.3004 - val_accuracy: 0.543